In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os
from torch.utils import data
from wavenet import Wavenet
from transformData import x_mu_law_encode,y_mu_law_encode,mu_law_decode,onehot,cateToSignal
from readDataset import Dataset

In [2]:
sampleSize=32000#the length of the sample size
quantization_channels=256
sample_rate=16000
dilations=[2**i for i in range(9)]*5  #idea from wavenet, have more receptive field
residualDim=128 #
skipDim=512
shapeoftest = 190500
filterSize=3
resumefile='testac' # name of checkpoint
lossname='testacloss.txt' # name of loss file
continueTrain=False # whether use checkpoint
pad = np.sum(dilations) # padding for dilate convolutional layers
lossrecord=[]  #list for record loss
pad=0

    #            |----------------------------------------|     *residual*
    #            |                                        |
    #            |    |-- conv -- tanh --|                |
    # -> dilate -|----|                  * ----|-- 1x1 -- + -->	*input*
    #                 |-- conv -- sigm --|     |    ||
    #                                         1x1=residualDim
    #                                          |
    # ---------------------------------------> + ------------->	*skip=skipDim*
    image changed from https://github.com/vincentherrmann/pytorch-wavenet/blob/master/wavenet_model.py

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use specific GPU

In [4]:
use_cuda = torch.cuda.is_available() # whether have available GPU
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor') #set_default_tensor_type as cuda tensor
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} 

In [5]:
params = {'batch_size': 1,'shuffle': True,'num_workers': 1}
training_set = Dataset(['origin_mix'],['origin_vocal'],'./vsCorpus/','./vsCorpus/')
testing_set = Dataset(['pred_mix'],['pred_mix'],'./vsCorpus/','./vsCorpus/')
loadtr = data.DataLoader(training_set, **params) #pytorch dataloader, more faster than mine
loadval = data.DataLoader(testing_set, **params)

In [6]:
model = Wavenet(pad,skipDim,quantization_channels,residualDim,dilations).cuda()
criterion = nn.CrossEntropyLoss()
#in wavenet paper, they said crossentropyloss is far better than MSELoss
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
#use adam to train
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

In [7]:
if continueTrain:# if continueTrain, the program will find the checkpoints
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

In [ ]:
def val(xtrain,ytrain): #validation last 15 seconds of the audio.
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        idx = np.arange(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
        print(correct,'accurate')
        print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-start_time))
        
        listofpred = []
        for ind in range(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,sampleSize):
            output = model(xtrain[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/notexval.wav', ans, sample_rate)
        print('val stored done',time.time() - start_time)
        

def test(xtrain):# testing data
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for iloader,(xtest,_) in enumerate(loadval):
            listofpred = []
            for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
                output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/notexte.wav', ans, sample_rate)

            listofpred=[]
            for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
                output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/notextr.wav', ans, sample_rate)
            print('test stored done',time.time() - start_time)
    
def train(epoch):#training data, the audio except for last 15 seconds
    model.train()
    for iloader,(xtrain,ytrain) in enumerate(loadtr):
        idx = np.arange(pad,xtrain.shape[-1]-pad-11*sampleSize,16000)
        np.random.shuffle(idx)#random the starting points
        for i, ind in enumerate(idx):
            start_time = time.time()
            data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
            output = model(data)
            loss = criterion(output, target)
            lossrecord.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                    epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
            if i % 100 == 0:
                with open("./lossRecord/"+lossname, "w") as f:
                    for s in lossrecord:
                        f.write(str(s) +"\n")
                print('write finish')

        val(xtrain,ytrain)
        test(xtrain)
        state={'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}
        torch.save(state, './model/'+resumefile)

In [ ]:
for epoch in range(100000):
    train(epoch)

Train Epoch: 0 [0/214 (0%)] Loss:5.566586: , (1.864 sec/step)
write finish
Train Epoch: 0 [1/214 (0%)] Loss:5.516381: , (1.831 sec/step)
Train Epoch: 0 [2/214 (1%)] Loss:5.634708: , (1.838 sec/step)
Train Epoch: 0 [3/214 (1%)] Loss:5.480080: , (1.842 sec/step)
Train Epoch: 0 [4/214 (2%)] Loss:5.410698: , (1.834 sec/step)
Train Epoch: 0 [5/214 (2%)] Loss:5.411734: , (1.850 sec/step)
Train Epoch: 0 [6/214 (3%)] Loss:5.263039: , (1.838 sec/step)
Train Epoch: 0 [7/214 (3%)] Loss:5.223952: , (1.838 sec/step)
Train Epoch: 0 [8/214 (4%)] Loss:5.167505: , (1.841 sec/step)
Train Epoch: 0 [9/214 (4%)] Loss:5.183545: , (1.859 sec/step)
Train Epoch: 0 [10/214 (5%)] Loss:5.231439: , (1.849 sec/step)
Train Epoch: 0 [11/214 (5%)] Loss:5.240126: , (1.856 sec/step)
Train Epoch: 0 [12/214 (6%)] Loss:5.267186: , (1.855 sec/step)
Train Epoch: 0 [13/214 (6%)] Loss:5.194458: , (1.865 sec/step)
Train Epoch: 0 [14/214 (7%)] Loss:5.176863: , (1.868 sec/step)
Train Epoch: 0 [15/214 (7%)] Loss:5.090603: , (1.862

Train Epoch: 0 [128/214 (60%)] Loss:4.893009: , (1.851 sec/step)
Train Epoch: 0 [129/214 (60%)] Loss:4.942778: , (1.849 sec/step)
Train Epoch: 0 [130/214 (61%)] Loss:4.994743: , (1.849 sec/step)
Train Epoch: 0 [131/214 (61%)] Loss:4.764468: , (1.846 sec/step)
Train Epoch: 0 [132/214 (62%)] Loss:4.766168: , (1.845 sec/step)
Train Epoch: 0 [133/214 (62%)] Loss:4.715384: , (1.842 sec/step)
Train Epoch: 0 [134/214 (63%)] Loss:4.839019: , (1.843 sec/step)
Train Epoch: 0 [135/214 (63%)] Loss:4.801147: , (1.839 sec/step)
Train Epoch: 0 [136/214 (64%)] Loss:4.671033: , (1.838 sec/step)
Train Epoch: 0 [137/214 (64%)] Loss:5.035991: , (1.836 sec/step)
Train Epoch: 0 [138/214 (64%)] Loss:4.726390: , (1.844 sec/step)
Train Epoch: 0 [139/214 (65%)] Loss:4.889451: , (1.843 sec/step)
Train Epoch: 0 [140/214 (65%)] Loss:5.005937: , (1.842 sec/step)
Train Epoch: 0 [141/214 (66%)] Loss:4.880929: , (1.845 sec/step)
Train Epoch: 0 [142/214 (66%)] Loss:4.770217: , (1.835 sec/step)
Train Epoch: 0 [143/214 (

Train Epoch: 1 [39/214 (18%)] Loss:4.451097: , (1.856 sec/step)
Train Epoch: 1 [40/214 (19%)] Loss:4.728004: , (1.856 sec/step)
Train Epoch: 1 [41/214 (19%)] Loss:4.665916: , (1.859 sec/step)
Train Epoch: 1 [42/214 (20%)] Loss:4.629697: , (1.855 sec/step)
Train Epoch: 1 [43/214 (20%)] Loss:4.466376: , (1.851 sec/step)
Train Epoch: 1 [44/214 (21%)] Loss:4.820478: , (1.849 sec/step)
Train Epoch: 1 [45/214 (21%)] Loss:4.631013: , (1.847 sec/step)
Train Epoch: 1 [46/214 (21%)] Loss:4.585552: , (1.849 sec/step)
Train Epoch: 1 [47/214 (22%)] Loss:4.574106: , (1.842 sec/step)
Train Epoch: 1 [48/214 (22%)] Loss:4.618058: , (1.854 sec/step)
Train Epoch: 1 [49/214 (23%)] Loss:4.645564: , (1.844 sec/step)
Train Epoch: 1 [50/214 (23%)] Loss:4.445055: , (1.839 sec/step)
Train Epoch: 1 [51/214 (24%)] Loss:4.422851: , (1.837 sec/step)
Train Epoch: 1 [52/214 (24%)] Loss:4.290728: , (1.835 sec/step)
Train Epoch: 1 [53/214 (25%)] Loss:4.741102: , (1.846 sec/step)
Train Epoch: 1 [54/214 (25%)] Loss:4.477

Train Epoch: 1 [166/214 (78%)] Loss:4.580834: , (1.852 sec/step)
Train Epoch: 1 [167/214 (78%)] Loss:4.665667: , (1.853 sec/step)
Train Epoch: 1 [168/214 (79%)] Loss:4.358455: , (1.856 sec/step)
Train Epoch: 1 [169/214 (79%)] Loss:4.483765: , (1.852 sec/step)
Train Epoch: 1 [170/214 (79%)] Loss:4.903052: , (1.858 sec/step)
Train Epoch: 1 [171/214 (80%)] Loss:4.540727: , (1.850 sec/step)
Train Epoch: 1 [172/214 (80%)] Loss:4.675546: , (1.842 sec/step)
Train Epoch: 1 [173/214 (81%)] Loss:4.561025: , (1.848 sec/step)
Train Epoch: 1 [174/214 (81%)] Loss:4.614818: , (1.840 sec/step)
Train Epoch: 1 [175/214 (82%)] Loss:4.454752: , (1.839 sec/step)
Train Epoch: 1 [176/214 (82%)] Loss:4.408684: , (1.840 sec/step)
Train Epoch: 1 [177/214 (83%)] Loss:4.594599: , (1.839 sec/step)
Train Epoch: 1 [178/214 (83%)] Loss:4.504374: , (1.839 sec/step)
Train Epoch: 1 [179/214 (84%)] Loss:4.701473: , (1.844 sec/step)
Train Epoch: 1 [180/214 (84%)] Loss:4.355359: , (1.843 sec/step)
Train Epoch: 1 [181/214 (

Train Epoch: 2 [78/214 (36%)] Loss:4.444396: , (1.846 sec/step)
Train Epoch: 2 [79/214 (37%)] Loss:4.427609: , (1.844 sec/step)
Train Epoch: 2 [80/214 (37%)] Loss:4.408062: , (1.844 sec/step)
Train Epoch: 2 [81/214 (38%)] Loss:4.270308: , (1.842 sec/step)
Train Epoch: 2 [82/214 (38%)] Loss:4.571140: , (1.841 sec/step)
Train Epoch: 2 [83/214 (39%)] Loss:4.386820: , (1.840 sec/step)
Train Epoch: 2 [84/214 (39%)] Loss:4.372581: , (1.852 sec/step)
Train Epoch: 2 [85/214 (40%)] Loss:4.516509: , (1.840 sec/step)
Train Epoch: 2 [86/214 (40%)] Loss:4.408685: , (1.839 sec/step)
Train Epoch: 2 [87/214 (41%)] Loss:4.337115: , (1.841 sec/step)
Train Epoch: 2 [88/214 (41%)] Loss:4.463127: , (1.842 sec/step)
Train Epoch: 2 [89/214 (42%)] Loss:4.386558: , (1.838 sec/step)
Train Epoch: 2 [90/214 (42%)] Loss:4.469464: , (1.842 sec/step)
Train Epoch: 2 [91/214 (43%)] Loss:4.430193: , (1.840 sec/step)
Train Epoch: 2 [92/214 (43%)] Loss:4.678338: , (1.841 sec/step)
Train Epoch: 2 [93/214 (43%)] Loss:4.209

...

0.03228125 accurate

val set:loss12.9710:, (0.588 sec/step)

val stored done 5.894796371459961
test stored done 73.86554098129272
Train Epoch: 211 [0/214 (0%)] Loss:0.891987: , (1.850 sec/step)
write finish
Train Epoch: 211 [1/214 (0%)] Loss:0.883398: , (1.851 sec/step)
Train Epoch: 211 [2/214 (1%)] Loss:0.802720: , (1.852 sec/step)
Train Epoch: 211 [3/214 (1%)] Loss:0.800329: , (1.852 sec/step)
Train Epoch: 211 [4/214 (2%)] Loss:0.776235: , (1.850 sec/step)
Train Epoch: 211 [5/214 (2%)] Loss:0.756827: , (1.851 sec/step)
Train Epoch: 211 [6/214 (3%)] Loss:1.024402: , (1.852 sec/step)
Train Epoch: 211 [7/214 (3%)] Loss:0.994523: , (1.852 sec/step)
Train Epoch: 211 [8/214 (4%)] Loss:0.783950: , (1.851 sec/step)
Train Epoch: 211 [9/214 (4%)] Loss:0.685404: , (1.851 sec/step)
Train Epoch: 211 [10/214 (5%)] Loss:0.952683: , (1.849 sec/step)
Train Epoch: 211 [11/214 (5%)] Loss:0.936089: , (1.849 sec/step)
Train Epoch: 211 [12/214 (6%)] Loss:1.069465: , (1.847 sec/step)
Train Epoch: 211 [13/2

Train Epoch: 211 [122/214 (57%)] Loss:0.990529: , (1.849 sec/step)
Train Epoch: 211 [123/214 (57%)] Loss:1.751229: , (1.849 sec/step)
Train Epoch: 211 [124/214 (58%)] Loss:0.986667: , (1.847 sec/step)
Train Epoch: 211 [125/214 (58%)] Loss:0.867771: , (1.847 sec/step)
Train Epoch: 211 [126/214 (59%)] Loss:0.765908: , (1.846 sec/step)
Train Epoch: 211 [127/214 (59%)] Loss:1.278035: , (1.843 sec/step)
Train Epoch: 211 [128/214 (60%)] Loss:0.868671: , (1.844 sec/step)
Train Epoch: 211 [129/214 (60%)] Loss:1.646725: , (1.844 sec/step)
Train Epoch: 211 [130/214 (61%)] Loss:1.625932: , (1.845 sec/step)
Train Epoch: 211 [131/214 (61%)] Loss:0.984111: , (1.846 sec/step)
Train Epoch: 211 [132/214 (62%)] Loss:1.071631: , (1.849 sec/step)
Train Epoch: 211 [133/214 (62%)] Loss:1.238872: , (1.850 sec/step)
Train Epoch: 211 [134/214 (63%)] Loss:1.102973: , (1.850 sec/step)
Train Epoch: 211 [135/214 (63%)] Loss:0.965509: , (1.850 sec/step)
Train Epoch: 211 [136/214 (64%)] Loss:1.529743: , (1.851 sec/s

Train Epoch: 212 [29/214 (14%)] Loss:0.894345: , (1.850 sec/step)
Train Epoch: 212 [30/214 (14%)] Loss:1.217868: , (1.853 sec/step)
Train Epoch: 212 [31/214 (14%)] Loss:1.065155: , (1.851 sec/step)
Train Epoch: 212 [32/214 (15%)] Loss:0.869824: , (1.856 sec/step)
Train Epoch: 212 [33/214 (15%)] Loss:1.004506: , (1.865 sec/step)
Train Epoch: 212 [34/214 (16%)] Loss:1.435915: , (1.853 sec/step)
Train Epoch: 212 [35/214 (16%)] Loss:1.015969: , (1.855 sec/step)
Train Epoch: 212 [36/214 (17%)] Loss:0.968995: , (1.856 sec/step)
Train Epoch: 212 [37/214 (17%)] Loss:1.124100: , (1.857 sec/step)
Train Epoch: 212 [38/214 (18%)] Loss:0.849453: , (1.866 sec/step)
Train Epoch: 212 [39/214 (18%)] Loss:0.986442: , (1.864 sec/step)
Train Epoch: 212 [40/214 (19%)] Loss:1.186129: , (1.860 sec/step)
Train Epoch: 212 [41/214 (19%)] Loss:1.196126: , (1.859 sec/step)
Train Epoch: 212 [42/214 (20%)] Loss:1.283110: , (1.862 sec/step)
Train Epoch: 212 [43/214 (20%)] Loss:1.407593: , (1.861 sec/step)
Train Epoc

Train Epoch: 212 [153/214 (71%)] Loss:1.021342: , (1.851 sec/step)
Train Epoch: 212 [154/214 (72%)] Loss:0.951117: , (1.850 sec/step)
Train Epoch: 212 [155/214 (72%)] Loss:1.396840: , (1.851 sec/step)
Train Epoch: 212 [156/214 (73%)] Loss:1.628430: , (1.850 sec/step)
Train Epoch: 212 [157/214 (73%)] Loss:1.649107: , (1.850 sec/step)
Train Epoch: 212 [158/214 (74%)] Loss:1.333695: , (1.848 sec/step)
Train Epoch: 212 [159/214 (74%)] Loss:2.070262: , (1.849 sec/step)
Train Epoch: 212 [160/214 (75%)] Loss:1.298790: , (1.848 sec/step)
Train Epoch: 212 [161/214 (75%)] Loss:1.024819: , (1.849 sec/step)
Train Epoch: 212 [162/214 (76%)] Loss:1.016134: , (1.850 sec/step)
Train Epoch: 212 [163/214 (76%)] Loss:1.114731: , (1.854 sec/step)
Train Epoch: 212 [164/214 (77%)] Loss:1.192720: , (1.850 sec/step)
Train Epoch: 212 [165/214 (77%)] Loss:1.346702: , (1.851 sec/step)
Train Epoch: 212 [166/214 (78%)] Loss:1.345434: , (1.850 sec/step)
Train Epoch: 212 [167/214 (78%)] Loss:1.265519: , (1.850 sec/s

Train Epoch: 213 [61/214 (29%)] Loss:2.770609: , (1.849 sec/step)
Train Epoch: 213 [62/214 (29%)] Loss:1.051451: , (1.848 sec/step)
Train Epoch: 213 [63/214 (29%)] Loss:1.743912: , (1.848 sec/step)
Train Epoch: 213 [64/214 (30%)] Loss:1.441107: , (1.846 sec/step)
Train Epoch: 213 [65/214 (30%)] Loss:1.361110: , (1.846 sec/step)
Train Epoch: 213 [66/214 (31%)] Loss:1.658802: , (1.845 sec/step)
Train Epoch: 213 [67/214 (31%)] Loss:1.432214: , (1.844 sec/step)
Train Epoch: 213 [68/214 (32%)] Loss:1.285105: , (1.844 sec/step)
Train Epoch: 213 [69/214 (32%)] Loss:1.763708: , (1.843 sec/step)
Train Epoch: 213 [70/214 (33%)] Loss:1.258254: , (1.842 sec/step)
Train Epoch: 213 [71/214 (33%)] Loss:1.586771: , (1.846 sec/step)
Train Epoch: 213 [72/214 (34%)] Loss:1.251958: , (1.840 sec/step)
Train Epoch: 213 [73/214 (34%)] Loss:1.335144: , (1.838 sec/step)
Train Epoch: 213 [74/214 (35%)] Loss:1.148855: , (1.840 sec/step)
Train Epoch: 213 [75/214 (35%)] Loss:1.033921: , (1.840 sec/step)
Train Epoc